In [ ]:
!pip install --upgrade pip
!pip install openAI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [1]:
# 혈액형 성격 요약 챗봇
from google.colab import userdata
from openai import OpenAI

openai_api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=openai_api_key)

user_input = input("혈액형을 입력하세요(예시: A형, B형, AB형, O형): ")

response = client.responses.create(
    model="gpt-5",
    input=[
        {
            "role": "user",
            "content": f"사용자가 입력한 혈액형 {user_input}에 해당하는 사람들의 특징과 성격을 3문장 이내로 알려줘."
        }
    ]
)

print("\n챗봇: ", response.output_text)
print("\n=== 혈액형 성격 챗봇이 종료되었습니다 ===")

혈액형을 입력하세요(예시: A형, B형, AB형, O형): O형

챗봇:  혈액형과 성격의 연관성은 과학적으로 확실하지 않지만, 대중적으로 O형은 대범하고 사교적이며 리더십이 있고 자신감과 추진력이 강하다고 여겨집니다. 반면에 승부욕이 강해 다소 고집스럽거나 자기주장이 강하게 보일 수 있다는 이미지도 있습니다.

=== 혈액형 성격 챗봇이 종료되었습니다 ===


In [2]:
# 스마트팜 어시스턴트
import random

from google.colab import userdata
from openai import OpenAI

openai_api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=openai_api_key)

def generate_sensor_data():
    """랜덤한 센서값 1세트를 생성"""
    data = {
        "temperature": round(random.uniform(15.0, 35.0), 2),  # °C
        "humidity": round(random.uniform(30.0, 90.0), 2),     # %
        "co2": round(random.uniform(400.0, 1000.0), 2),       # ppm
        "ec": round(random.uniform(0.5, 3.0), 2),             # dS/m (전기전도도)
        "ph": round(random.uniform(5.5, 7.5), 2)              # pH
    }
    return data

def assistant_by_current_data(data: dict, model: str = "gpt-5") -> str:
    """
    현재 센서값(data)을 바탕으로, 이상적인 스마트팜 제어환경을 자연어 문장으로 제시.
    출력 예시:
    "현재 온도는 29.4°C로 다소 높습니다. 온도를 25°C 근처로 낮추기 위해 환기팬을 작동하는 것이 좋습니다..."
    """
    SYSTEM_INSTRUCTIONS = (
        "너는 스마트팜 환경 제어 전문가다. "
        "대한민국 전라남도 전라남도 곡성 옥과면에서 스마트팜으로 레몬을 재배하고 있다. "
        "입력으로 주어진 센서값을 기반으로 현재 상태를 간단히 진단하고, "
        "현재 날짜를 기준으로 이상적인 제어 환경(온도·습도·EC·pH·CO2 목표 범위와 권장 제어 조치)을 3~5문장으로 설명하라. "
        "JSON이나 목록 대신 자연스러운 문장으로만 답하라."
    )

    # 입력 텍스트 구성
    user_text = (
        f"현재 센서값은 다음과 같습니다:\n"
        f"온도: {data.get('temperature', 'N/A')}°C, "
        f"습도: {data.get('humidity', 'N/A')}%, "
        f"EC: {data.get('ec', 'N/A')} dS/m, "
        f"pH: {data.get('ph', 'N/A')}, "
        f"CO₂: {data.get('co2', 'N/A')} ppm.\n\n"
        "이 값을 바탕으로 이상적인 제어환경과 권장 조치를 설명해주세요."
    )

    response = client.responses.create(
        model=model,
        instructions=SYSTEM_INSTRUCTIONS,
        input=[
            {
                "role": "user",
                "content": [
                    {"type": "input_text", "text": user_text}
                ]
            }
        ]
    )

    return (response.output_text or "").strip()

if __name__ == "__main__":
    sensor_data = generate_sensor_data()
    print("=== 스마트팜 장소: 전라남도 곡성 옥과면 ===")
    print("=== 스마트팜 작목: 레몬 ===")
    print(sensor_data)

    advice = assistant_by_current_data(sensor_data, model="gpt-5")
    print("\n=== 스마트팜 환경 제어 조언 ===")
    print(advice)

=== 스마트팜 장소: 전라남도 곡성 옥과면 ===
=== 스마트팜 작목: 레몬 ===
{'temperature': 19.75, 'humidity': 60.16, 'co2': 998.52, 'ec': 1.63, 'ph': 7.21}

=== 스마트팜 환경 제어 조언 ===
현재 상태는 주간이라면 온도가 다소 낮고 야간이라면 무난한 편이며, 습도는 적정 범위, EC는 양호하지만 pH 7.21은 레몬에 다소 알칼리성이라 미량원소 흡수 저해 위험이 있고 CO₂는 광합성 시간대 기준으로 적절합니다. 지금 시기(10월 말) 전남 곡성의 레몬 하우스 이상치는 주간 21~24°C, 야간 14~18°C, 상대습도 55~65%, 양액 EC 1.6~1.8 dS/m, pH 5.8~6.3, CO₂는 채광 시 800~1000 ppm입니다. 권장 조치는 주간 난방으로 온도를 1~3°C 올리고 제습·환기 균형을 맞춰 55~65%를 유지하며, EC는 현 수준을 유지하되 원수 알칼리도 점검 후 질산·인산 등으로 pH를 약 6.0까지 천천히 낮추는 것입니다. CO₂는 채광 시간에만 주입하고 과습이나 과다 CO₂ 시 환기로 조정하며, pH 교정 후 철·망간 결핍 증상(엽맥간 황화 등)이 완화되는지 모니터링하세요.


In [3]:
import gradio as gr

from google.colab import userdata
from openai import OpenAI

openai_api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=openai_api_key)

def assistant_by_current_data(
    temperature: float,
    humidity: float,
    ec: float,
    ph: float,
    co2: float,
    model: str = "gpt-5",
) -> str:
    """
    현재 센서값을 바탕으로 이상적인 스마트팜 제어환경을 자연어로 제시
    """

    SYSTEM_INSTRUCTIONS = (
        "너는 스마트팜 환경 제어 전문가다. "
        "대한민국 전라남도 곡성군 옥과면에서 스마트팜으로 레몬을 재배하고 있다. "
        "입력으로 주어진 센서값을 기반으로 현재 상태를 간단히 진단하고, "
        "현재 날짜를 기준으로 이상적인 제어 환경(온도·습도·EC·pH·CO₂ 목표 범위와 권장 제어 조치)을 3~5문장으로 설명하라. "
        "JSON이나 목록 대신 자연스러운 문장으로만 답하라."
    )

    user_text = (
        f"현재 센서값은 다음과 같습니다:\n"
        f"온도: {temperature}°C, "
        f"습도: {humidity}%, "
        f"EC: {ec} dS/m, "
        f"pH: {ph}, "
        f"CO₂: {co2} ppm.\n\n"
        "이 값을 바탕으로 이상적인 제어환경과 권장 조치를 설명해주세요."
    )

    response = client.responses.create(
        model=model,
        instructions=SYSTEM_INSTRUCTIONS,
        input=[
            {
                "role": "user",
                "content": [{"type": "input_text", "text": user_text}],
            }
        ],
    )
    return (response.output_text or "").strip()

# Gradio UI
with gr.Blocks(title="스마트팜 센서 입력 + 제어 조언") as demo:
    gr.Markdown("## 🌿 스마트팜 센서 입력 & 제어 조언 (OpenAI Responses API)")
    gr.Markdown(
        "- 슬라이더 또는 숫자 중 편한 방식으로 값을 설정하세요.\n"
        "- 환경변수를 입력한 후 **조언 받기**를 누르세요."
    )

    with gr.Row():
        model = gr.Dropdown(
            label="모델 선택",
            choices=["gpt-4o-mini", "gpt-4o", "gpt-5"],
            value="gpt-4o-mini",
        )

    with gr.Row():
        with gr.Column():
            gr.Markdown("### 🎚️ 슬라이더 입력")
            s_temp = gr.Slider(-20.0, 60.0, value=25.0, step=0.1, label="온도 (°C)")
            s_hum  = gr.Slider(0.0, 100.0, value=60.0, step=0.5, label="습도 (%)")
            s_ec   = gr.Slider(0.0, 10.0, value=1.5, step=0.01, label="EC (dS/m)")
            s_ph   = gr.Slider(0.0, 10.0, value=6.5, step=0.01, label="pH")
            s_co2  = gr.Slider(0.0, 1500.0, value=600.0, step=10.0, label="CO₂ (ppm)")
        with gr.Column():
            gr.Markdown("### 💡 스마트팜 환경 제어 조언")
            advice_out = gr.Textbox(label=" ", lines=13, show_copy_button=True)

        # --- 조언 받기 버튼 ---
    btn_advice = gr.Button("💡 조언 받기", variant="primary")

    btn_advice.click(
        fn=assistant_by_current_data,
        inputs=[s_temp, s_hum, s_ec, s_ph, s_co2, model],
        outputs=advice_out,
    )


if __name__ == "__main__":
    demo.queue().launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5e2ed8b6f3166cfd8c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
